# Lab: Metaclass

Write a metaclass that allows us to use `class` syntax to create a `dict`, e.g.

```python
class MyDict(object, metaclass=Dictify):
    a = 1
    b = 2
```

creates the dict `{'a': 1, 'b': 2}`

Hint: Metaclasses and class decorators are not required to return classes!

In [2]:
class Dictify(type):
    def __new__(meta, name, bases, dct):
        return {
            k: v for k, v in dct.items()
            if not k.startswith('__')
        }
    
class MyDict(object, metaclass=Dictify):
    a = 1
    b = 2
    c = 'foo'
    
MyDict

{'a': 1, 'b': 2, 'c': 'foo'}

# Lab: Descriptor

Write a (non-data) descriptor that will allow us to calculate an attribute value the *first* time it is loaded, and cache it for subsequent loads.

In [9]:
sentry = object()

class reify():
    def __init__(self, getter):
        self._getter = getter
        self._value = sentry
        
    def __get__(self, instance, type):
        if instance is None:
            return self
        if self._value is sentry:
            self._value = self._getter(instance)
        return self._value

class MyClass():
    
    @reify
    def a(self):
        print('Calculate a!')
        return 'a'

In [4]:
obj = MyClass()
obj.a

Calculate a!


'a'

In [5]:
obj.a

'a'

Even better, we can use the fact that this is a *non-data* descriptor:

In [10]:
class reify():
    def __init__(self, getter):
        self._getter = getter
        
    def __get__(self, instance, type):
        if instance is None:
            return self
        value = self._getter(instance)
        instance.__dict__[self._getter.__name__] = value
        return value

class MyClass2():
    
    @reify
    def a(self):
        print('Calculate a!')
        return 'a'

In [11]:
obj = MyClass2()
obj.a

Calculate a!


'a'

In [12]:
obj.a

'a'

In [13]:
obj1 = MyClass()
obj1.a

Calculate a!


'a'

In [14]:
obj2 = MyClass2()
obj2.a

Calculate a!


'a'

In [15]:
%timeit obj1.a

247 ns ± 5.71 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
%timeit obj2.a

55 ns ± 0.579 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
